In [1]:
from datamap import experiment_files
experiment_files.keys()

dict_keys(['pos_2022', 'neg_2022', 'pos_2020', 'neg_2020', 'remdesivir_33', 'ac4C_10', '2-OmeATP_10', 'remdesivir_0', 's4U_33', 's4U_5', 'm5C_10', 'm5C_5', '2-OmeATP_0', 's4U_0', 'm6A_33', 'ac4C_33', '2-OmeATP_5', 'remdesivir_5', 's4U_10', 'm5C_0', 'm6A_5', 'm6A_0', 'ac4C_0', 'm6A_10'])

In [ ]:
from torchmetrics import ConfusionMatrix
import torch
target = torch.tensor([1, 1, 1,0, 0,0,0])
preds = torch.tensor([0, 1,0,1,0, 0,0])
confmat = ConfusionMatrix(num_classes=2, normalize='true')
confmat(preds, target)


In [5]:
from datamap import experiment_files
len(list(experiment_files['pos_2022_fast'][0].iterdir()))

4000

In [7]:
from dataloading2 import nanopore_datamodule
dm = nanopore_datamodule(pos_files = 'pos_2022_fast', neg_files='pos_2022_fast', verbose=True)

valid files indicies
[14, 34, 36, 41, 46, 87, 116, 189, 193, 254]
[14, 41, 68, 71, 87, 165, 189, 203, 228, 254]
train files indicies
[54, 68, 71, 165, 203, 228, 229]
[34, 36, 46, 54, 116, 193, 229]


In [1]:
import pytorch_lightning as pl
from torch import nn
import torch
from torch.nn import functional as F
from torch.nn import RNN
import torchmetrics
from resnet1d.resnet1d import ResNet1D




/home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jovyan/my-conda-envs/RNAmodEnv/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")


In [11]:
from transformers import BertModel, BertConfig
BertModel(BertConfig())

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [1]:
from tst import Transformer
from transformer.src.benchmark import LSTM, BiGRU, ConvGru, FullyConv, FFN
import torch
# LSTM(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# BiGRU(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# ConvGru(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
# FullyConv(input_dim=1, hidden_dim=64, output_dim=1)
# FFN(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)

#TODO chunk=none??
#TODO attention size
leng = 1000
model = Transformer(d_input=1, d_model=1, d_output=1,input_length=1000, q=8,v=8,h=8, N=4, attention_size=50, chunk_mode=None, pe='regular', skip_embedding=True)
# print(model)
print(model(torch.rand(32,1000,1)).size())
print(model)

SKIPPING EMBEDDING, d_iput and d_model must be equal
torch.Size([32, 1])
Transformer(
  (layers_encoding): ModuleList(
    (0): Encoder(
      (_selfAttention): MultiHeadAttention(
        (_W_q): Linear(in_features=1, out_features=64, bias=True)
        (_W_k): Linear(in_features=1, out_features=64, bias=True)
        (_W_v): Linear(in_features=1, out_features=64, bias=True)
        (_W_o): Linear(in_features=64, out_features=1, bias=True)
      )
      (_feedForward): PositionwiseFeedForward(
        (_linear1): Linear(in_features=1, out_features=2048, bias=True)
        (_linear2): Linear(in_features=2048, out_features=1, bias=True)
      )
      (_layerNorm1): LayerNorm((1,), eps=1e-05, elementwise_affine=True)
      (_layerNorm2): LayerNorm((1,), eps=1e-05, elementwise_affine=True)
      (_dopout): Dropout(p=0.3, inplace=False)
    )
    (1): Encoder(
      (_selfAttention): MultiHeadAttention(
        (_W_q): Linear(in_features=1, out_features=64, bias=True)
        (_W_k): Linea

In [45]:
class GenericModel(pl.LightningModule):
    def __init__(self, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate


        self.net = LSTM(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # BiGRU(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # ConvGru(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)
        # FullyConv(input_dim=1, hidden_dim=64, output_dim=1)
        # FFN(input_dim=1, hidden_dim=64, output_dim=1, num_layers=3)

        self.acc = torchmetrics.Accuracy()

    def forward(self, x):
        out = self.net.forward(x)
        return out[:,-1,:]
  
      

    def configure_optimizers(self):
      # print("LEARNING RATE:",self.learning_rate)
      optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=0.01) #wd 0.01
      return optimizer

    def training_step(self, train_batch, batch_idx):
      x,y = train_batch
      output = self(x)
      loss = F.binary_cross_entropy_with_logits(output, y)
      self.log('train_loss', loss)
      acc =self.acc(output, y.int())
      self.log('train acc', acc)
      return loss

    def validation_step(self, val_batch, batch_idx):
      x,y = val_batch
      output = self(x)
      loss = F.binary_cross_entropy_with_logits(output, y)
      self.log('valid_loss', loss)
      acc = self.acc(output, y.int())
      self.log('valid acc', acc)

rnn = GenericModel(0.001)
rnn(torch.swapaxes(torch.rand(32,1,1000),-1,-2)).size()


torch.Size([32, 1])